In [3]:
import tensorflow as tf
import tensorflow_compression as tfc
import tensorflow_probability as tfp
tfd = tfp.distributions

tf.enable_eager_execution()

In [4]:
tfc.GDN

tensorflow_compression.python.layers.gdn.GDN